In [26]:
import sys
from typing import Any

import cv2 as cv
import numpy as np

In [27]:

def merge_overlapping_detections(detections: list, overlap_threshold: float = 0.3) -> list:
    """
    Merges overlapping bounding boxes based on Intersection over Union (IoU).

    Parameters:
        detections: List of bounding boxes to merge.
        overlap_threshold: IoU threshold for merging boxes. Boxes with IoU >= threshold are merged.

    Returns:
        list: List of merged bounding boxes.
    """
    if not detections:
        return []

    boxes = np.array(detections)
    x1, y1 = boxes[:, 0], boxes[:, 1]
    x2, y2 = boxes[:, 2], boxes[:, 3]
    areas = (x2 - x1 + 1) * (y2 - y1 + 1)

    # Sort boxes by their area in descending order
    order = areas.argsort()[::-1]
    merged_boxes = []

    while len(order) > 0:
        i = order[0]
        merged_boxes.append(boxes[i])

        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])

        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        inter = w * h
        union = areas[i] + areas[order[1:]] - inter
        iou = inter / union

        # Keep boxes with IoU below the threshold
        remain_indices = np.where(iou < overlap_threshold)[0] + 1
        order = order[remain_indices]

    return merged_boxes

In [28]:
def detect_moving_objects(frame: np.ndarray, background_subtractor: cv.BackgroundSubtractor,
                          area_threshold: int = 100) -> tuple[Any, Any]:
    """
    Detects moving objects using a background subtractor, returns their bounding boxes.

    Parameters:
        frame: Current video frame.
        background_subtractor: Background subtractor for motion detection.
        area_threshold: Minimum area for detected bounding boxes.


    Returns:
        List of detected bounding boxes with their areas, [x1, y1, x2, y2, area]
    """

    fg_mask = background_subtractor.apply(frame)
    contours, _ = cv.findContours(fg_mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    detections = []
    for cnt in contours:
        x, y, w, h = cv.boundingRect(cnt)
        area = w * h
        if area > area_threshold:
            detections.append([x, y, x + w, y + h, area])

    return detections, fg_mask

In [29]:
def process_video(video_path: str, scale: float = 0.5, overlap_threshold: float = 0.3, area_threshold: int = 100):
    """
    Processes the input video.

    Parameters:
        video_path: Path to the video file.
        scale: Scaling factor for resizing frames.
        overlap_threshold: Threshold for merging overlapping detections using IoU.
        area_threshold: Minimum area for detected bounding boxes.

    """
    capture = cv.VideoCapture(video_path)
    back_sub = cv.createBackgroundSubtractorMOG2(detectShadows=False)

    while True:
        ret, frame = capture.read()
        if not ret:
            break

        frame = cv.resize(frame, None, fx=scale, fy=scale, interpolation=cv.INTER_LINEAR)
        detections, _ = detect_moving_objects(frame, back_sub, area_threshold=area_threshold)
        merged_detections = merge_overlapping_detections(detections, overlap_threshold)

        # Draw bounding boxes
        for det in merged_detections:
            x1, y1, x2, y2, _ = det
            cv.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        cv.imshow('Frame', frame)
        key = cv.waitKey(1)
        if key in [27, ord('q'), ord('Q')]:
            break

    capture.release()
    cv.destroyAllWindows()

    if sys.platform == 'darwin':
        for _ in range(4):
            cv.waitKey(1)

    cv.destroyAllWindows()



# Main

In [30]:
video_path = 'SamsungGear360.mp4'
process_video(video_path, scale=0.5, overlap_threshold=0.0005, area_threshold=700)